# Tutorial 1: Text Pre-processing and Wrangling

Text is different than usual datasets we use to build our typical Machine Learning models. Text data needs to be pre-processed to ensure we have it in a form that is usable for various NLP tasks. Text processing and wrangling is a necessary and important step in any NLP project.

In this notebook, we will cover:
- Sentence tokenization
- Word tokenization
- Handling non-text characters - accents, special symbols, HTML
- Preprocessing steps such as spell correction, stemming, lemmatization, expanding contractions and stopword removal

### Import Libraries

In [1]:
import re
import nltk
import requests
import numpy as np
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('europarl_raw')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package europarl_raw to /root/nltk_data...
[nltk_data]   Unzipping corpora/europarl_raw.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Get Text Dataset

Project Gutenberg is a large opensource and free collection of literary works from across the world. In this case, we will leverage content of the book **The Bible - Book 1: Genesis** for understanding different text processing and wrangling steps in the following sections


We will also use a smaller sample text with a few short sentences to demostrate examples as well.

In [3]:
bible_html_url = "http://www.gutenberg.org/cache/epub/8001/pg8001.html"
bible_txt_url = "http://www.gutenberg.org/cache/epub/8001/pg8001.txt"
data = requests.get(bible_txt_url)
content = data.text

In [15]:
print(content[970:1800])





Title: The Bible, King James version, Book 1: Genesis

Release Date: May, 2005  [EBook #8001]
[Yes, we are more than one year ahead of schedule]
[This file was first posted on June 7, 2003]


Edition: 10

Language: English


*** START OF THE PROJECT GUTENBERG EBOOK, THE BIBLE, KING JAMES, BOOK 1***




This eBook was produced by David Widger
with the help of Derek Andrew's text from January 1992
and the work of Bryan Taylor in November 2002.





Book 01        Genesis

01:001:001 In the beginning God created the heaven and the earth.

01:001:002 And the earth was without form, and void; and darkness was
           upon the face of the deep. And the Spirit of God moved upon
           the face of the waters.

01:001:003 And God said, Let there be light: and there was light.

01


In [16]:
# Total characters in War and Peace
len(content)

262238

In [17]:
sample_text = ("US unveils world's most powerful supercomputer, beats China. " 
               "The US has unveiled the world's most powerful supercomputer called 'Summit', " 
               "beating the previous record-holder China's Sunway TaihuLight. With a peak performance "
               "of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, "
               "which is capable of 93,000 trillion calculations per second. Summit has 4,608 servers, "
               "which reportedly take up the size of two tennis courts.")
sample_text

"US unveils world's most powerful supercomputer, beats China. The US has unveiled the world's most powerful supercomputer called 'Summit', beating the previous record-holder China's Sunway TaihuLight. With a peak performance of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, which is capable of 93,000 trillion calculations per second. Summit has 4,608 servers, which reportedly take up the size of two tennis courts."

In [18]:
# First 100 characters in the corpus
content[0:100]

'\ufeffProject Gutenberg EBook The Bible, King James, Book 1: Genesis\r\n\r\nCopyright laws are changing all o'

## Sentence Tokenization

Sentence is a syntactical as well as a logical division of content in a given corpus. In order to understand text or prepare it for various tasks, understanding sentence boundaries is an important step.

Sentence tokenization is the process of determining sentence boundaries in a given corpus.

One might think that it is merely trivial to determine a sentence boundary, we just need to split based on ".". While this generally holds true yet there are exceptions. Think of scenarios where we use "." in abbreviations and shorthand notations(such as Mr. or Mrs.).


Let us now go through a few standard ways of performing sentence tokenization

### NLTK's Default Tokenizer

NLTK provides a number of sentence tokenizers. Let's first have a look at the default one.

In [20]:
default_st = nltk.sent_tokenize
bib_sentences = default_st(text=content)
sample_sentences = default_st(text=sample_text)

print('Total sentences in sample_text:{}'.format(len(sample_sentences)))
print('Sample text sentences :-')
print(np.array(sample_sentences))

print('\nTotal sentences in bible:', len(bib_sentences))
print('First 5 sentences in bible:-')
print(np.array(bib_sentences[0:5]))

Total sentences in sample_text:4
Sample text sentences :-
["US unveils world's most powerful supercomputer, beats China."
 "The US has unveiled the world's most powerful supercomputer called 'Summit', beating the previous record-holder China's Sunway TaihuLight."
 'With a peak performance of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, which is capable of 93,000 trillion calculations per second.'
 'Summit has 4,608 servers, which reportedly take up the size of two tennis courts.']

Total sentences in bible: 1560
First 5 sentences in bible:-
['\ufeffProject Gutenberg EBook The Bible, King James, Book 1: Genesis\r\n\r\nCopyright laws are changing all over the world.'
 'Be sure to check the\r\ncopyright laws for your country before downloading or redistributing\r\nthis or any other Project Gutenberg eBook.'
 'This header should be the first thing seen when viewing this Project\r\nGutenberg file.'
 'Please do not remove it.'
 'Do not change or ed

### Punkt Tokenizer

This tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. 

In [21]:
punkt_st = nltk.tokenize.PunktSentenceTokenizer()
sample_sentences = punkt_st.tokenize(sample_text)
print(np.array(sample_sentences))

["US unveils world's most powerful supercomputer, beats China."
 "The US has unveiled the world's most powerful supercomputer called 'Summit', beating the previous record-holder China's Sunway TaihuLight."
 'With a peak performance of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, which is capable of 93,000 trillion calculations per second.'
 'Summit has 4,608 servers, which reportedly take up the size of two tennis courts.']


### Regex Tokenizer

A RegexpTokenizer splits a string into substrings using predefined patterns (needs to be defined by the user)

In [22]:
SENTENCE_TOKENS_PATTERN = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z]\.)(?<=\.|\?|\!)\s'
regex_st = nltk.tokenize.RegexpTokenizer(
            pattern=SENTENCE_TOKENS_PATTERN,
            gaps=True)
sample_sentences = regex_st.tokenize(sample_text)
print(np.array(sample_sentences))

["US unveils world's most powerful supercomputer, beats China."
 "The US has unveiled the world's most powerful supercomputer called 'Summit', beating the previous record-holder China's Sunway TaihuLight."
 'With a peak performance of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, which is capable of 93,000 trillion calculations per second.'
 'Summit has 4,608 servers, which reportedly take up the size of two tennis courts.']


### Tokenize German Sentences

NLTK also provides utilities to handle sentence tokenization for various languages (apart from English). The following is a sample to showcase sentence tokenization for German text

In [23]:
from nltk.corpus import europarl_raw

german_text = europarl_raw.german.raw(fileids='ep-00-01-17.de')
# Total characters in the corpus
print(len(german_text))
# First 100 characters in the corpus
print(german_text[0:100])

157171
 
Wiederaufnahme der Sitzungsperiode Ich erkläre die am Freitag , dem 17. Dezember unterbrochene Sit


In [24]:
# default sentence tokenizer 
german_sentences_def = default_st(text=german_text, language='german')

# loading german text tokenizer into a PunktSentenceTokenizer instance  
german_tokenizer = nltk.data.load(resource_url='tokenizers/punkt/german.pickle')
german_sentences = german_tokenizer.tokenize(german_text)

In [25]:
# check if results of both tokenizers match 
# should be True
print(german_sentences_def == german_sentences)

True


In [26]:
# print first 5 sentences of the corpus
print(np.array(german_sentences[:5]))

[' \nWiederaufnahme der Sitzungsperiode Ich erkläre die am Freitag , dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen , wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe , daß Sie schöne Ferien hatten .'
 'Wie Sie feststellen konnten , ist der gefürchtete " Millenium-Bug " nicht eingetreten .'
 'Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden .'
 'Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen .'
 'Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen - , allen Opfern der Stürme , insbesondere in den verschiedenen Ländern der Europäischen Union , in einer Schweigeminute zu gedenken .']


## Word Tokenization

Word can be considered as a basic building block for NLP tasks. A combination of words make up a sentence. As in the previous section, we worked towards understanding sentence tokenization, in this section, we will focus on understanding word boundaries.

We will make use of various word tokenizers available from ``nltk`` as well as ``spacy`` in this section

In [27]:
# default tokenizer
default_wt = nltk.word_tokenize
words = default_wt(sample_text)
np.array(words)

array(['US', 'unveils', 'world', "'s", 'most', 'powerful',
       'supercomputer', ',', 'beats', 'China', '.', 'The', 'US', 'has',
       'unveiled', 'the', 'world', "'s", 'most', 'powerful',
       'supercomputer', 'called', "'Summit", "'", ',', 'beating', 'the',
       'previous', 'record-holder', 'China', "'s", 'Sunway', 'TaihuLight',
       '.', 'With', 'a', 'peak', 'performance', 'of', '200,000',
       'trillion', 'calculations', 'per', 'second', ',', 'it', 'is',
       'over', 'twice', 'as', 'fast', 'as', 'Sunway', 'TaihuLight', ',',
       'which', 'is', 'capable', 'of', '93,000', 'trillion',
       'calculations', 'per', 'second', '.', 'Summit', 'has', '4,608',
       'servers', ',', 'which', 'reportedly', 'take', 'up', 'the', 'size',
       'of', 'two', 'tennis', 'courts', '.'], dtype='<U13')

### Treebank Tokenizer

The Treebank tokenizer uses regular expressions to tokenize text as in [Penn Treebank](https://web.archive.org/web/19970614072242if_/http://www.cis.upenn.edu/~treebank/tokenization.html).

In [29]:
treebank_wt = nltk.TreebankWordTokenizer()
words = treebank_wt.tokenize(sample_text)
np.array(words)

array(['US', 'unveils', 'world', "'s", 'most', 'powerful',
       'supercomputer', ',', 'beats', 'China.', 'The', 'US', 'has',
       'unveiled', 'the', 'world', "'s", 'most', 'powerful',
       'supercomputer', 'called', "'Summit", "'", ',', 'beating', 'the',
       'previous', 'record-holder', 'China', "'s", 'Sunway',
       'TaihuLight.', 'With', 'a', 'peak', 'performance', 'of', '200,000',
       'trillion', 'calculations', 'per', 'second', ',', 'it', 'is',
       'over', 'twice', 'as', 'fast', 'as', 'Sunway', 'TaihuLight', ',',
       'which', 'is', 'capable', 'of', '93,000', 'trillion',
       'calculations', 'per', 'second.', 'Summit', 'has', '4,608',
       'servers', ',', 'which', 'reportedly', 'take', 'up', 'the', 'size',
       'of', 'two', 'tennis', 'courts', '.'], dtype='<U13')

### TokTok Tokenizer

The tok-tok tokenizer is a simple, general tokenizer, where the input has one sentence per line; thus only final period is tokenized.

Tok-tok has been tested on, and gives reasonably good results for English, Persian, Russian, Czech, French, German, Vietnamese, Tajik, and a few others. The input should be in UTF-8 encoding.



In [30]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
words = tokenizer.tokenize(sample_text)
np.array(words)

array(['US', 'unveils', 'world', "'", 's', 'most', 'powerful',
       'supercomputer', ',', 'beats', 'China.', 'The', 'US', 'has',
       'unveiled', 'the', 'world', "'", 's', 'most', 'powerful',
       'supercomputer', 'called', "'", 'Summit', "'", ',', 'beating',
       'the', 'previous', 'record-holder', 'China', "'", 's', 'Sunway',
       'TaihuLight.', 'With', 'a', 'peak', 'performance', 'of', '200,000',
       'trillion', 'calculations', 'per', 'second', ',', 'it', 'is',
       'over', 'twice', 'as', 'fast', 'as', 'Sunway', 'TaihuLight', ',',
       'which', 'is', 'capable', 'of', '93,000', 'trillion',
       'calculations', 'per', 'second.', 'Summit', 'has', '4,608',
       'servers', ',', 'which', 'reportedly', 'take', 'up', 'the', 'size',
       'of', 'two', 'tennis', 'courts', '.'], dtype='<U13')

### Custom Tokenizer

``nltk`` also allows us to create our own custom tokenizers using regular expressions, whitespaces, etc.

In [31]:
# regex based custom tokenizer-1
GAP_PATTERN = r'\s+'        
regex_wt = nltk.RegexpTokenizer(pattern=GAP_PATTERN,
                                gaps=True)
words = regex_wt.tokenize(sample_text)
np.array(words)

array(['US', 'unveils', "world's", 'most', 'powerful', 'supercomputer,',
       'beats', 'China.', 'The', 'US', 'has', 'unveiled', 'the',
       "world's", 'most', 'powerful', 'supercomputer', 'called',
       "'Summit',", 'beating', 'the', 'previous', 'record-holder',
       "China's", 'Sunway', 'TaihuLight.', 'With', 'a', 'peak',
       'performance', 'of', '200,000', 'trillion', 'calculations', 'per',
       'second,', 'it', 'is', 'over', 'twice', 'as', 'fast', 'as',
       'Sunway', 'TaihuLight,', 'which', 'is', 'capable', 'of', '93,000',
       'trillion', 'calculations', 'per', 'second.', 'Summit', 'has',
       '4,608', 'servers,', 'which', 'reportedly', 'take', 'up', 'the',
       'size', 'of', 'two', 'tennis', 'courts.'], dtype='<U14')

In [32]:
# whitespace tokenizer
whitespace_wt = nltk.WhitespaceTokenizer()
words = whitespace_wt.tokenize(sample_text)
np.array(words)

array(['US', 'unveils', "world's", 'most', 'powerful', 'supercomputer,',
       'beats', 'China.', 'The', 'US', 'has', 'unveiled', 'the',
       "world's", 'most', 'powerful', 'supercomputer', 'called',
       "'Summit',", 'beating', 'the', 'previous', 'record-holder',
       "China's", 'Sunway', 'TaihuLight.', 'With', 'a', 'peak',
       'performance', 'of', '200,000', 'trillion', 'calculations', 'per',
       'second,', 'it', 'is', 'over', 'twice', 'as', 'fast', 'as',
       'Sunway', 'TaihuLight,', 'which', 'is', 'capable', 'of', '93,000',
       'trillion', 'calculations', 'per', 'second.', 'Summit', 'has',
       '4,608', 'servers,', 'which', 'reportedly', 'take', 'up', 'the',
       'size', 'of', 'two', 'tennis', 'courts.'], dtype='<U14')

In [33]:
# utility for tokenization
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences] 
    return word_tokens

sents = tokenize_text(sample_text)
np.array(sents)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


array([list(['US', 'unveils', 'world', "'s", 'most', 'powerful', 'supercomputer', ',', 'beats', 'China', '.']),
       list(['The', 'US', 'has', 'unveiled', 'the', 'world', "'s", 'most', 'powerful', 'supercomputer', 'called', "'Summit", "'", ',', 'beating', 'the', 'previous', 'record-holder', 'China', "'s", 'Sunway', 'TaihuLight', '.']),
       list(['With', 'a', 'peak', 'performance', 'of', '200,000', 'trillion', 'calculations', 'per', 'second', ',', 'it', 'is', 'over', 'twice', 'as', 'fast', 'as', 'Sunway', 'TaihuLight', ',', 'which', 'is', 'capable', 'of', '93,000', 'trillion', 'calculations', 'per', 'second', '.']),
       list(['Summit', 'has', '4,608', 'servers', ',', 'which', 'reportedly', 'take', 'up', 'the', 'size', 'of', 'two', 'tennis', 'courts', '.'])],
      dtype=object)

In [34]:
words = [word for sentence in sents for word in sentence]
np.array(words)

array(['US', 'unveils', 'world', "'s", 'most', 'powerful',
       'supercomputer', ',', 'beats', 'China', '.', 'The', 'US', 'has',
       'unveiled', 'the', 'world', "'s", 'most', 'powerful',
       'supercomputer', 'called', "'Summit", "'", ',', 'beating', 'the',
       'previous', 'record-holder', 'China', "'s", 'Sunway', 'TaihuLight',
       '.', 'With', 'a', 'peak', 'performance', 'of', '200,000',
       'trillion', 'calculations', 'per', 'second', ',', 'it', 'is',
       'over', 'twice', 'as', 'fast', 'as', 'Sunway', 'TaihuLight', ',',
       'which', 'is', 'capable', 'of', '93,000', 'trillion',
       'calculations', 'per', 'second', '.', 'Summit', 'has', '4,608',
       'servers', ',', 'which', 'reportedly', 'take', 'up', 'the', 'size',
       'of', 'two', 'tennis', 'courts', '.'], dtype='<U13')

### Spacy Tokenizer

``spacy`` provides easy to use interfaces to perform sentence and word tokenization.

In [35]:
import spacy

In [36]:
nlp = spacy.load('en', parse = True, tag=True, entity=True)

text_spacy = nlp(sample_text)

In [37]:
sents = np.array(list(text_spacy.sents))
sents

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([US unveils world's most powerful supercomputer, beats China.,
       The US has unveiled the world's most powerful supercomputer called 'Summit', beating the previous record-holder China's Sunway TaihuLight.,
       With a peak performance of 200,000 trillion calculations per second, it is over twice as fast as Sunway TaihuLight, which is capable of 93,000 trillion calculations per second.,
       Summit has 4,608 servers, which reportedly take up the size of two tennis courts.],
      dtype=object)

In [38]:
sent_words = [[word.text for word in sent] for sent in sents]
np.array(sent_words)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


array([list(['US', 'unveils', 'world', "'s", 'most', 'powerful', 'supercomputer', ',', 'beats', 'China', '.']),
       list(['The', 'US', 'has', 'unveiled', 'the', 'world', "'s", 'most', 'powerful', 'supercomputer', 'called', "'", 'Summit', "'", ',', 'beating', 'the', 'previous', 'record', '-', 'holder', 'China', "'s", 'Sunway', 'TaihuLight', '.']),
       list(['With', 'a', 'peak', 'performance', 'of', '200,000', 'trillion', 'calculations', 'per', 'second', ',', 'it', 'is', 'over', 'twice', 'as', 'fast', 'as', 'Sunway', 'TaihuLight', ',', 'which', 'is', 'capable', 'of', '93,000', 'trillion', 'calculations', 'per', 'second', '.']),
       list(['Summit', 'has', '4,608', 'servers', ',', 'which', 'reportedly', 'take', 'up', 'the', 'size', 'of', 'two', 'tennis', 'courts', '.'])],
      dtype=object)

In [39]:
words = [word.text for word in text_spacy]
np.array(words)

array(['US', 'unveils', 'world', "'s", 'most', 'powerful',
       'supercomputer', ',', 'beats', 'China', '.', 'The', 'US', 'has',
       'unveiled', 'the', 'world', "'s", 'most', 'powerful',
       'supercomputer', 'called', "'", 'Summit', "'", ',', 'beating',
       'the', 'previous', 'record', '-', 'holder', 'China', "'s",
       'Sunway', 'TaihuLight', '.', 'With', 'a', 'peak', 'performance',
       'of', '200,000', 'trillion', 'calculations', 'per', 'second', ',',
       'it', 'is', 'over', 'twice', 'as', 'fast', 'as', 'Sunway',
       'TaihuLight', ',', 'which', 'is', 'capable', 'of', '93,000',
       'trillion', 'calculations', 'per', 'second', '.', 'Summit', 'has',
       '4,608', 'servers', ',', 'which', 'reportedly', 'take', 'up',
       'the', 'size', 'of', 'two', 'tennis', 'courts', '.'], dtype='<U13')

## Handling Non-Text Characters

Natural text consists of various types of characters such as alphabets, numbers, symbols, emoticons, non-printable characters and so on. For most practical use-cases we limit ourselves to alphabets (at most numbers) and ignore other types of characters. 

In this section, we will focus on identification of non-text characters and how to remove them from our corpus safely.

We will focus on handling following types of characters:
- Accented Characters
- Special Characters
- HTML Tags & Noise


### Accented Characters

The most common accents are the acute (é), grave (è), circumflex (â, î or ô), tilde (ñ), umlaut and dieresis (ü or ï – the same symbol is used for two different purposes), and cedilla (ç). Accent marks (also referred to as diacritics or diacriticals) usually appear above a character.

These characters are part of extended alphabet in languages such as French, Spanish, etc. 

In [40]:
import unicodedata

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [42]:
s = 'Sómě Áccěntěd těxt'
s

'Sómě Áccěntěd těxt'

In [43]:
remove_accented_chars(s)

'Some Accented text'

### Special Characters

Symbols, emoticons and characters such as ``#``, ``@`` etc. are considered special characters

In [44]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [45]:
s = "Well this was fun! See you at 7:30, What do you think!!? #$@@9318@ 🙂🙂🙂"
s

'Well this was fun! See you at 7:30, What do you think!!? #$@@9318@ 🙂🙂🙂'

In [46]:
remove_special_characters(s, remove_digits=True)

'Well this was fun See you at  What do you think  '

In [47]:
remove_special_characters(s)

'Well this was fun See you at 730 What do you think 9318 '

### HTML Tags & Noise

Many times, NLP datasets are collected as part of web-scraping activities. Web-scraping involves scanning various websites to extract text from them. This process leads to content which is a mix of actual text as well as HTML tags.

In this section we will extract HTML version of ** The Bible** book. We will then use ``BeautifulSoup`` to clean out HTML tags to get actual text.

In [49]:
data = requests.get(bible_html_url)
content = data.text
print(content[2745:3948])


<p id="id00011" style="margin-top: 2em">*** START OF THE PROJECT GUTENBERG EBOOK, THE BIBLE, KING JAMES, BOOK 1***</p>

<p id="id00012" style="margin-top: 4em">This eBook was produced by David Widger
with the help of Derek Andrew's text from January 1992
and the work of Bryan Taylor in November 2002.</p>

<h1 id="id00013" style="margin-top: 5em">Book 01        Genesis</h1>

<p id="id00014">01:001:001 In the beginning God created the heaven and the earth.</p>

<p id="id00015" style="margin-left: 0%; margin-right: 0%">01:001:002 And the earth was without form, and void; and darkness was
           upon the face of the deep. And the Spirit of God moved upon
           the face of the waters.</p>

<p id="id00016">01:001:003 And God said, Let there be light: and there was light.</p>

<p id="id00017">01:001:004 And God saw the light, that it was good: and God divided the<br/>

           light from the darkness.<br/>
</p>

<p id="id00018">01:001:005 And God called the light Day, and the dar

In [50]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

In [51]:
clean_content = strip_html_tags(content)
print(clean_content[1163:1957])

*** START OF THE PROJECT GUTENBERG EBOOK, THE BIBLE, KING JAMES, BOOK 1***
This eBook was produced by David Widger
with the help of Derek Andrew's text from January 1992
and the work of Bryan Taylor in November 2002.
Book 01        Genesis
01:001:001 In the beginning God created the heaven and the earth.
01:001:002 And the earth was without form, and void; and darkness was
           upon the face of the deep. And the Spirit of God moved upon
           the face of the waters.
01:001:003 And God said, Let there be light: and there was light.
01:001:004 And God saw the light, that it was good: and God divided the
           light from the darkness.
01:001:005 And God called the light Day, and the darkness he called
           Night. And the evening and the morning were the first day.




That seemed to have worked like a charm!
---

## Text Correction

In this section, we will prepare utilities to fix different issues with textual data.

- Fix Repeat Characters
- Spell Correction
- Expand Contractions
- Stemming
- Lemmatization

### Repeat Characters

Repeating characters are a common mistake in textual data. We will look at different techniques to handle the same.

In [52]:
old_word = 'finalllyyy'

#### Base Version: Regex pattern to search repeating characters

In [53]:
repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
match_substitution = r'\1\2\3'

In [54]:
step = 1

while True:
    # remove one repeated character
    new_word = repeat_pattern.sub(match_substitution,
                                  old_word)
    if new_word != old_word:
         print('Step: {} Word: {}'.format(step, new_word))
         step += 1 # update step
         # update old word to last substituted state
         old_word = new_word  
         continue
    else:
         print("Final word:", new_word)
         break

Step: 1 Word: finalllyy
Step: 2 Word: finallly
Step: 3 Word: finally
Step: 4 Word: finaly
Final word: finaly


#### Enhanced Version: Use Wordnet to check if new word is a dictionary word to stop repeated text removal at the right point.

In [55]:
from nltk.corpus import wordnet

In [56]:
old_word = 'finalllyyy'

In [57]:
repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
match_substitution = r'\1\2\3'

In [58]:
step = 1
 
while True:
    # check for semantically correct word
    if wordnet.synsets(old_word):
        print("Final correct word:", old_word)
        break
    # remove one repeated character
    new_word = repeat_pattern.sub(match_substitution,
                                  old_word)
    if new_word != old_word:
        print('Step: {} Word: {}'.format(step, new_word))
        step += 1 # update step
        # update old word to last substituted state
        old_word = new_word  
        continue
    else:
        print("Final word:", new_word)
        break

Step: 1 Word: finalllyy
Step: 2 Word: finallly
Step: 3 Word: finally
Final correct word: finally


#### Optimized Version: Use regex and wordnet to perform cleanup

In [59]:
def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
            
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens

In [60]:
sample_sentence = 'This tutorial is realllllyyy amaaazingggg'
correct_tokens = remove_repeated_characters(nltk.word_tokenize(sample_sentence))
' '.join(correct_tokens)

'This tutorial is really amazing'

_Note:_ This may fail for some edge cases based on order of word removals, spell check is an option after this to get the correct word.

### Spell Correction

#### Base Version:
1. Prepare a list of words along with their frequency (you can use any corpora for this having correct spelled out words, more the better. The corpora we use here is the same also referenced in _Chapter 3 of Text Analytics with Python 2nd ed._)
2. Prepare utility functions to calculate edit-distance of 0, 1 and 2 based on possible replacements of individual characters in a given word
3. Search for replacements in the word-frequency list for possible corrections

In [84]:
spell_corpus_resp = requests.get('https://github.com/dipanjanS/text-analytics-with-python/blob/master/New-Second-Edition/Ch03%20-%20Processing%20and%20Understanding%20Text/big.txt?raw=true')
spell_corpus = spell_corpus_resp.text

In [85]:
spell_corpus[:100]

'The Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in o'

In [86]:
import collections

In [87]:
def tokens(text): 
    """
    Get all words from the corpus
    """
    return re.findall('[a-z]+', text.lower()) 

In [88]:
WORDS = tokens(spell_corpus)
WORD_COUNTS = collections.Counter(WORDS)
# top 10 words in corpus
WORD_COUNTS.most_common(10)

[('the', 80030),
 ('of', 40025),
 ('and', 38313),
 ('to', 28766),
 ('in', 22050),
 ('a', 21155),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [89]:
 def edits0(word): 
    """
    Return all strings that are zero edits away 
    from the input word (i.e., the word itself).
    """
    return {word}



def edits1(word):
    """
    Return all strings that are one edit away 
    from the input word.
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    def splits(word):
        """
        Return a list of all possible (first, rest) pairs 
        that the input word is made of.
        """
        return [(word[:i], word[i:]) 
                for i in range(len(word)+1)]
                
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    """Return all strings that are two edits away 
    from the input word.
    """
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [90]:
def known(words):
    """
    Return the subset of words that are actually 
    in our WORD_COUNTS dictionary.
    """
    return {w for w in words if w in WORD_COUNTS}

In [91]:
# input word
In [409]: word = 'fianlly'

# zero edit distance from input word
edits0(word)

{'fianlly'}

In [92]:
# returns null set since it is not a valid word
known(edits0(word))

set()

In [93]:
# one edit distance from input word
list(edits1(word))[:10]

['fianlmy',
 'fiqnlly',
 'fiaxlly',
 'sianlly',
 'ofianlly',
 'fianldly',
 'yianlly',
 'fdanlly',
 'fianlldy',
 'fbianlly']

In [94]:
# get correct words from above set
known(edits1(word))

{'finally'}

In [95]:
# two edit distances from input word
list(edits2(word))[:10]

['fianljlm',
 'fianlxgy',
 'fwiranlly',
 'tianllk',
 'sfcianlly',
 'fhanllty',
 'fiaubly',
 'fialnllyp',
 'piadnlly',
 'fiancwy']

In [96]:
# get correct words from above set
known(edits2(word))

{'faintly', 'finally', 'finely', 'frankly'}

In [97]:
candidates = (known(edits0(word)) or 
              known(edits1(word)) or 
              known(edits2(word)) or 
              [word])
candidates

{'finally'}

In [98]:
def correct(word):
    """
    Get the best correct spelling for the input word
    """
    # Priority is for edit distance 0, then 1, then 2
    # else defaults to the input word itself.
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=WORD_COUNTS.get)

In [99]:
correct('fianlly')

'finally'

#### Improved Version:
Use ``textblob`` to find correct word

In [100]:
from textblob import Word

w = Word('fianlly')
w.correct()

'finally'

In [101]:
w.spellcheck()

[('finally', 1.0)]

In [102]:
w = Word('flaot')
w.spellcheck()

[('flat', 0.85), ('float', 0.15)]

## Stemming

In linguistic morphology and information retrieval, stemming is the process of reducing inflected words to their word stem, base or root form—generally a written word form

#### Porter Stemmer

The Porter stemming algorithm (or 'Porter stemmer') is a process for removing the commoner morphological and inflexional endings from words in English.

In [103]:
from nltk.stem import PorterStemmer

In [104]:
ps = PorterStemmer()

ps.stem('jumping'), ps.stem('jumps'), ps.stem('jumped')

('jump', 'jump', 'jump')

In [105]:
ps.stem('lying')

'lie'

In [106]:
ps.stem('strange')

'strang'

#### Lancaster Stemmer
The Lancaster stemmers are more aggressive and dynamic compared to the other two stemmers. The stemmer is really faster, but the algorithm is really confusing when dealing with small words. But they are not as efficient as Snowball Stemmers. The Lancaster stemmers save the rules externally and basically uses an iterative algorithm.

In [107]:
from nltk.stem import LancasterStemmer
ls = LancasterStemmer()

ls.stem('jumping'), ls.stem('jumps'), ls.stem('jumped')

('jump', 'jump', 'jump')

In [108]:
ls.stem('lying')

'lying'

In [109]:
ls.stem('strange')

'strange'

#### Regex based stemmer

Regular Expression based Stemmer

In [110]:
from nltk.stem import RegexpStemmer
rs = RegexpStemmer('ing$|s$|ed$', min=4)
rs.stem('jumping'), rs.stem('jumps'), rs.stem('jumped')

('jump', 'jump', 'jump')

In [111]:
rs.stem('lying')

'ly'

#### Snowball Stemmer

Snowball is a small string processing language for creating stemming algorithms for use in Information Retrieval, plus a collection of stemming algorithms implemented using it.


In [112]:
# Snowball Stemmer
from nltk.stem import SnowballStemmer
ss = SnowballStemmer("german")
print('Supported Languages:', SnowballStemmer.languages)

Supported Languages: ('arabic', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'porter', 'portuguese', 'romanian', 'russian', 'spanish', 'swedish')


In [113]:
# stemming on German words
# autobahnen -> cars
# autobahn -> car
ss.stem('autobahnen')

'autobahn'

In [116]:
ps = nltk.porter.PorterStemmer()
ls = nltk.stem.LancasterStemmer()

def simple_stemmer(text, stemmer=ps):
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

#### Try calling the above defined function for both Lancaster and Porter stemmer separately

In [117]:
s = "My system keeps crashing his crashed yesterday ours crashes daily and presumably we are not lying"
s

'My system keeps crashing his crashed yesterday ours crashes daily and presumably we are not lying'

In [118]:
simple_stemmer(s, stemmer=ps)

'My system keep crash hi crash yesterday our crash daili and presum we are not lie'

In [119]:
simple_stemmer(s, stemmer=ls)

'my system keep crash his crash yesterday our crash dai and presum we ar not lying'

## Lemmatization

Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma

In [120]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [121]:
# lemmatize nouns
print(wnl.lemmatize('cars', 'n'))
print(wnl.lemmatize('men', 'n'))

car
men


In [122]:
# lemmatize verbs
print(wnl.lemmatize('running', 'v'))
print(wnl.lemmatize('ate', 'v'))

run
eat


In [123]:
# lemmatize adjectives
print(wnl.lemmatize('saddest', 'a'))
print(wnl.lemmatize('fancier', 'a'))

sad
fancy


In [124]:
# ineffective lemmatization
print(wnl.lemmatize('ate', 'n'))
print(wnl.lemmatize('fancier', 'v'))
print(wnl.lemmatize('fancier'))

ate
fancier
fancier


#### Building your own lemmatizer using nltk

Define a function such that you put all the above steps together so that it does the following
- Function name is `wordnet_lemmatize_text(...)`
- Input is a variable text which should take in a document (bunch of words)
- Need to tokenize the text
- Get POS tags of tokenized text
- Convert POS tags into wordnet (single letter) POS tags
- use nltk's wordnet lemmatizer
- Return lemmatized text as the output (as a string)

In [125]:
from nltk.corpus import wordnet
wnl = WordNetLemmatizer()

def wordnet_lemmatize_text(text):
  # tokenize text
  tokens = nltk.word_tokenize(text)
  # pos tag tokenized text
  tagged_tokens = nltk.pos_tag(tokens)
  # convert raw POS tags into wordnet tags
  tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
  # treat unknown tags as nouns by default
  new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), 
                                          wordnet.NOUN))
                            for word, tag in tagged_tokens]

  lemmatized_text = ' '.join(wnl.lemmatize(word, tag) for word, tag in new_tagged_tokens)
  return lemmatized_text

In [126]:
s = 'The brown foxes are quick and they are jumping over the sleeping lazy dogs!'
s

'The brown foxes are quick and they are jumping over the sleeping lazy dogs!'

In [127]:
wordnet_lemmatize_text(s)

'The brown fox be quick and they be jump over the sleep lazy dog !'

#### Spacy Lemmatization

Out of the box implementation

In [129]:
import spacy
# use spacy.load('en') if you have downloaded the language model en directly after install spacy
nlp = spacy.load('en')

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [130]:
lemmatize_text(s)

'the brown fox be quick and they be jump over the sleep lazy dog !'

## Stopword Removal

In computing, stop words are words which are filtered out before or after processing of natural language data. A stop word is a commonly used word (such as “the”, “a”, “an”,etc.) which does not convey a lot of useful information

We typically remove stopwords before using text for most NLP tasks

In [131]:
def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [132]:
stop_words = nltk.corpus.stopwords.words('english')
print(stop_words[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [133]:
s

'The brown foxes are quick and they are jumping over the sleeping lazy dogs!'

In [134]:
remove_stopwords(s, is_lower_case=False)

'brown foxes quick jumping sleeping lazy dogs !'

Remove the word 'the' and add the word 'brown' from the stop_words list and call the function with this new list

In [135]:
stop_words.remove('the')
stop_words.append('brown')

In [136]:
remove_stopwords(s, is_lower_case=False, stopwords=stop_words)

'The foxes quick jumping the sleeping lazy dogs !'

## Expand Contractions

Contractions are words or combinations of words that are shortened by dropping letters and replacing them by an apostrophe.

In order to capture context better, we standardize text by expanding such contractions. ``contractions`` and ``textsearch`` enable us to do so in just a few lines of code

In [137]:
!pip install contractions
!pip install textsearch

     |████████████████████████████████| 245kB 8.5MB/s 
     |████████████████████████████████| 317kB 8.3MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=37d2f38e31c8e3dc63cf032e427c5f2c4a1e103cf6d82f6c3b2b88dbab4a6b89
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [138]:
import contractions

In [139]:
list(contractions.contractions_dict.items())[:10]

[("I'm", 'I am'),
 ("I'm'a", 'I am about to'),
 ("I'm'o", 'I am going to'),
 ("I've", 'I have'),
 ("I'll", 'I will'),
 ("I'll've", 'I will have'),
 ("I'd", 'I would'),
 ("I'd've", 'I would have'),
 ('Whatcha', 'What are you'),
 ("amn't", 'am not')]

In [141]:
sample_string = "Y'all can't expand contractions I'd think! You wouldn't be able to. How'd you do it?"
sample_string

"Y'all can't expand contractions I'd think! You wouldn't be able to. How'd you do it?"

In [142]:
contractions.fix(sample_string)

'you all can not expand contractions I would think! You would not be able to. how did you do it?'